### ETL - ARQUIVOS EM DIRETÓRIO



[Documentação do pymssql](https://pypi.org/project/pymssql/)

#### Objetivo:

- Extrair todos os arquivos dentro de um diretório com a extensão .xlsx
- Consolidar os dados
- Inputar em uma tabela no banco de dados SQL Server

In [1]:
import os
import pandas as pd
import pymssql as sql

In [2]:
#caminho onde os arquivos estão
path = 'C:/Users/Lenovo/Desktop/ESTOQUE/BASE'
#definindo o diretório
try:
    os.chdir(path) #seta o diretório
except FileExistsError: #caso exista
    print("Diretporio incorreto!")
else:
    os.chdir(path) #use o diretório

#lista os arquivos
arquivos = os.listdir()

In [3]:
#consolidando os dados
lista_arquivo = []

for pasta in arquivos:
    if pasta.endswith(".xlsx"): #filtro por extenção de arquivos
        lista_arquivo.append(pd.read_excel(pasta)) #inserindo arquivos na lista 
    
df = pd.concat(lista_arquivo, ignore_index=True, axis=0)

In [4]:
#Setando a conexão com SQL Server
SERVER = 'localhost'
DATABASE = 'DW_ESTOQUE'
USERNAME = 'DESKTOP-C66TR5R\Lenovo'
PASSWORD = ''
#criando conexção com SQL Server. Parâmetros: (servidor, usuario, senha, database)
conexao = sql.connect(server=SERVER, user= '', password='', database=DATABASE)
#cursor de conexão
cursor = conexao.cursor()

In [5]:
#looping insert no banco de  dados
for lin in range(df.index.max() + 1):
    Id_Produto = df.loc[lin][0]
    Nome_Produto = df.loc[lin][1]
    Categoria = df.loc[lin][2]
    Data_Entrada = df.loc[lin][3] if pd.notna(df.loc[lin][3]) else None  # Verificar se a data existe
    Data_Saida = df.loc[lin][4] if pd.notna(df.loc[lin][4]) else None
    Local_Estoque = df.loc[lin][5]
    Estoque_Inicial = df.loc[lin][6] if pd.notna(df.loc[lin][6]) else 0  # Definir valor padrão
    Entradas = df.loc[lin][7] if pd.notna(df.loc[lin][7]) else 0
    Saidas = df.loc[lin][8] if pd.notna(df.loc[lin][8]) else 0
    Quebras = df.loc[lin][9] if pd.notna(df.loc[lin][9]) else 0
    Perdas = df.loc[lin][10] if pd.notna(df.loc[lin][10]) else 0
    Estoque_Final = df.loc[lin][11] if pd.notna(df.loc[lin][11]) else 0
    Tempo_Vida_Produto = df.loc[lin][12] if pd.notna(df.loc[lin][12]) else 0
    Data_Periodo = df.loc[lin][13] if pd.notna(df.loc[lin][13]) else None
    Loja = df.loc[lin][14]
    #inserindo valores no BD
    insert_comando = f"""
    INSERT INTO [DW_ESTOQUE].[dbo].[STG_ESTOQUE] (ID_Produto, Nome_Produto, Categoria, Data_Entrada, Data_Saida, Local_Estoque, Estoque_Inicial, Entradas, Saidas, Quebras, Perdas, Estoque_Final, Tempo_Vida_Produto, Data_Periodo, Loja) 
    VALUES
    ('{Id_Produto}', '{Nome_Produto}','{Categoria}', '{Data_Entrada}', '{Data_Saida}', '{Local_Estoque}', {Estoque_Inicial}, {Entradas}, {Saidas}, {Quebras}, {Perdas}, {Estoque_Final}, {Tempo_Vida_Produto}, '{Data_Periodo}', '{Loja}')"""

    cursor.execute(insert_comando)
    #pprint(insert_comando)

cursor.close()
conexao.close()

In [6]:
#Consultar dados no Sql Server
#sql = "SELECT TOP 10 * FROM [DW_ESTOQUE].[dbo].[STG_ESTOQUE]"
# Executando a query no banco de dados
#cursor.execute(sql)
# Recuperando os resultados
#resultados = cursor.fetchall()

# Imprimindo os resultados
#for linha in resultados:
#    print(linha)
#insert_comando
#cursor.close()
#conexao.close()